In [39]:
#转换地名为拼音格式
import pypinyin

list0 =["常德","渭南","孝感","丽水","运城","德州","张家口","鄂尔多斯","阳江","泸州","丹东","曲靖","乐山","许昌","湘潭","晋中","安阳","齐齐哈尔","北海","宝鸡","抚州","景德镇","延安","三明","抚顺亳州","日照","衢州","拉萨","淮北","焦作","平顶山","滨州","吉安","濮阳","眉山","池州","荆门","铜仁","长治","衡水","铜陵","承德","达州","邵阳","德阳","龙岩","南平","淮南","黄石","营口","东营","吉林","韶关","枣庄","怀化","宣城","临汾","聊城","梅州","盘锦","锦州","榆林","玉林","十堰","汕尾","咸宁","宜宾","永州","益阳","黔南州","黔东南","恩施","红河","大理","大同","鄂州","忻州","吕梁","黄山","开封","郴州","茂名","漯河","葫芦岛","河源","娄底","延边"]
# ["银川","西宁","海口","洛阳","南通","常州","徐州","潍坊","淄博","绍兴","温州","台州","大庆","鞍山","中山","珠海","汕头","吉林","柳州","拉萨","保定","邯郸","秦皇岛","沧州","鄂尔多斯","东营","威海","济宁","临沂","德州","滨州","泰安","湖州","嘉兴","金华","泰州","镇江","盐城","扬州","桂林","惠州","湛江","江门","茂名","株洲","岳阳","衡阳","宝鸡","宜昌","襄樊","开封","许昌","平顶山","赣州","九江","芜湖","绵阳","齐齐哈尔","牡丹江","抚顺","本溪","丹东","辽阳","锦州","营口","承德","廊坊","邢台","大同","榆林","延安","天水","克拉玛依","喀什","石河子","南阳","濮阳","安阳","焦作","新乡","日照","聊城","枣庄","蚌埠","淮南","马鞍山","连云港","淮安","丽水","衢州","荆州","安庆","景德镇","新余","湘潭","常德","郴州","漳州","清远","揭阳","梅州","肇庆","玉林","北海","德阳","宜宾","遵义","大理"]
list_pinyin = []
for i in list0:
    s = ''
    for j in pypinyin.pinyin(i, style=pypinyin.NORMAL):
        s += ''.join(j)
    list_pinyin.append(s)

In [40]:
len(list_pinyin)

87

In [41]:
import requests,re,os
from lxml import etree
from fontTools.ttLib import TTFont
import xlwt
import webbrowser
import os
from tqdm import tqdm
import time
f = xlwt.Workbook()
# 创建一个sheet
sheet1 = f.add_sheet('new',cell_overwrite_ok=True)
col1 = sheet1.col(0)
col1.width = 256*38
#初始化第一行
row0 = [u'商铺名称',u'类型',u'评分',u'评论',u'人均消费',u'店铺地址',u'当地地址',u'效果',u'环境',u'服务',u'地点']
for i in range(0,len(row0)):
    sheet1.write(0,i,row0[i])#初始化第一行~

def download_woff(css_url):
    # 获取woff的url
    woff_url = re.findall(r',url\("(//s3plus.*?\.woff)"\);\}', requests.get(css_url).text, re.S)
    # 下载woff文件到本地
    for url in woff_url:
        content = requests.get('http:' + url).content
        filename = url.split('/')[-1]
        with open('./css破解/' + filename, 'wb') as fp:
            fp.write(content)
    return [name.split('/')[-1] for name in woff_url]

character = list(
    '1234567890店中美家馆小车大市公酒行国品发电金心业商司超生装园场食有新限天面工'
    '服海华水房饰城乐汽香部利子老艺花专东肉菜学福饭人百餐茶务通味所山区门药银农龙停尚安'
    '广鑫一容动南具源兴鲜记时机烤文康信果阳理锅宝达地儿衣特产西批坊州牛佳化五米修爱北养'
    '卖建材三会鸡室红站德王光名丽油院堂烧江社合星货型村自科快便日民营和活童明器烟育宾精'
    '屋经居庄石顺林尔县手厅销用好客火雅盛体旅之鞋辣作粉包楼校鱼平彩上吧保永万物教吃设医'
    '正造丰健点汤网庆技斯洗料配汇木缘加麻联卫川泰色世方寓风幼羊烫来高厂兰阿贝皮全女拉成'
    '云维贸道术运都口博河瑞宏京际路祥青镇厨培力惠连马鸿钢训影甲助窗布富牌头四多妆吉苑沙'
    '恒隆春干饼氏里二管诚制售嘉长轩杂副清计黄讯太鸭号街交与叉附近层旁对巷栋环省桥湖段乡'
    '厦府铺内侧元购前幢滨处向座下県凤港开关景泉塘放昌线湾政步宁解白田町溪十八古双胜本'
    '单同九迎第台玉锦底后七斜期武岭松角纪朝峰六振珠局岗洲横边济井办汉代临弄团外塔杨铁浦'
    '字年岛陵原梅进荣友虹央桂沿事津凯莲丁秀柳集紫旗张谷的是不了很还个也这我就在以可到错'
    '没去过感次要比觉看得说常真们但最喜哈么别位能较境非为欢然他挺着价那意种想出员两推做'
    '排实分间甜度起满给热完格荐喝等其再几只现朋候样直而买于般豆量选奶打每评少算又因情找'
    '些份置适什蛋师气你姐棒试总定啊足级整带虾如态且尝主话强当更板知己无酸让入啦式笑赞'
    '片酱差像提队走嫩才刚午接重串回晚微周值费性桌拍跟块调糕')
length = 0
for x in tqdm(range(0,len(list_pinyin))):
    time.sleep(0.1)
    url = 'http://www.dianping.com/'+ str(list_pinyin[x]) +'/ch10/g110' # url = 'http://www.dianping.com/changsha/ch10/g110'
    headers={
    'cookie':"_lxsdk_cuid=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _lxsdk=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _hc.v=ebb31d04-778f-02a6-9984-060d56d71344.1617504706; s_ViewType=10; ctu=3a87482384aa8445669918ae73680ee9a42215c28654ba91ce2d4ec9bf99ab35; ll=7fd06e815b796be3df069dec7836c3df; aburl=1; cityid=3; default_ab=index%3AA%3A3%7CshopList%3AA%3A5; switchcityflashtoast=1; cy=17; cye=xian; fspop=test; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1618070722,1618320955,1618320969,1618322864; ua=dpuser_0539703749; uamo=17390601491; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1618331052; _lxsdk_s=178cc01486f-3c-2b2-b9d%7C%7C28",
    'User-Agent':'Opera/9.80 (Android 2.3.4; Linux; Opera mobi/adr-1107051709; U; zh-cn) Presto/2.8.149 Version/11.10',
    'Referer':'http://www.dianping.com/changsha/ch10/g110',
    }
    #破解css
    
#     os.chdir("D://360downloads//美团比赛//css破解//") #设置需要查看的目录D:\360downloads\美团比赛\css破解
#     a = [x for x in os.listdir()]
#     for jack in a:
#         os.remove("D://360downloads//美团比赛//css破解//"+jack)

    rep=requests.get(url,headers=headers).text
    css_url='http:'+re.search(r'(//s3plus.*?\.css)',rep).group(1)
    css_dir='./css破解/'
    if not os.path.exists(css_dir):
        os.mkdir(css_dir)
    filenames=os.listdir(css_dir)
    if not filenames:
        #下载woff文件
        filenames=download_woff(css_url)
    data={}

    for filename in filenames:
        font=TTFont(css_dir+filename)
        keys=font.getGlyphOrder()[2:]
        for index,value in enumerate(keys):
            if value[3:] not in data:
                data[value[3:]]=character[index]
    # 5、利用data字典，进行内容替换
    for k, v in data.items():
        rep = rep.replace(f'&#x{k};', v)
    # tree=etree.HTML(rep)
    # li_list=tree.xpath('//div[@class="content"]/div/ul/li')
    # for li in li_list:
    #     title=li.xpath('.//div[@class="tit"]/a/h4/text()')[0]
    #     tag=li.xpath('string(.//div[@class="tag-addr"]/a/span)')
    #     print(title,tag)
    tree=etree.HTML(rep)
    li_list=tree.xpath('//div[@class="content"]/div/ul/li')
    count = 0
    for li in li_list:
        title = li.xpath('.//div[@class="tit"]/a/h4/text()')[0]
        tag = li.xpath('string(.//div[@class="tag-addr"]/a/span)')
        score = li.xpath(
            'string(.//div[@class="star_score score_50  star_score_sml"]/text()|.//div[@class="star_score score_40  star_score_sml"]/text()|.//div[@class="star_score score_45  star_score_sml"]/text())')
        comments = li.xpath('string(.//div[@class="comment"]/a[@class="review-num"]/b)')
        prize = li.xpath('string(.//div[@class="comment"]/a[@class="mean-price"]/b)')
        address = li.xpath('string(.//div[@class="tag-addr"]/a[@data-click-name="shop_tag_region_click"]/span)')
        localaddress = li.xpath('string(.//div[@class="tag-addr"]/span)')
        flavor = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=1])')
        environment = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=2])')
        service = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=3])')
        import re
        flavor = re.sub("口味", '',flavor)
        environment = re.sub("环境", '',environment)
        service = re.sub("服务", '',service)
        list1 = [comments, prize, flavor, environment, service]
        i = 0
        for i in range(0,len(list1)):
            k = list1[i]
            if "货" in k:
                k = re.sub("货", '8',k)
            if "旅" in k:
                k = re.sub("旅", '6',k)
            if "联" in k:
                k = re.sub("联", '9',k)
            list1[i] = k
            i = i + 1
        comments, prize, flavor, environment, service = list1[0],list1[1],list1[2],list1[3],list1[4]
#         print('\t',title,'\t',tag,'\t', score,'\t', comments,'\t', prize,'\t', address,'\t', localaddress,'\t', flavor,'\t', environment,'\t', service)
#         print("~~~"*10)
        a = [title, tag, score, comments, prize, address, localaddress, flavor, environment, service,list0[x]]
        for j in range(0,len(a)):
            sheet1.write(length + 1+count,j,a[j])
        count = count + 1
    length = length + count
f.save("大众点评收集数据.xls")
webbrowser.open('D:/360downloads/美团比赛/大众点评收集数据.xls')



  0%|                                                                                           | 0/87 [00:00<?, ?it/s]

  1%|▉                                                                                  | 1/87 [00:00<00:57,  1.50it/s]

  2%|█▉                                                                                 | 2/87 [00:01<00:55,  1.54it/s]

  3%|██▊                                                                                | 3/87 [00:01<00:53,  1.57it/s]

  5%|███▊                                                                               | 4/87 [00:02<00:52,  1.58it/s]

  6%|████▊                                                                              | 5/87 [00:03<00:52,  1.57it/s]

  7%|█████▋                                                                             | 6/87 [00:03<00:52,  1.55it/s]

  8%|██████▋                                                                            | 7/87 [00:04<00:51,  1.55it/s]

  9%|███████▋                 

 74%|████████████████████████████████████████████████████████████▎                     | 64/87 [00:40<00:14,  1.59it/s]

 75%|█████████████████████████████████████████████████████████████▎                    | 65/87 [00:41<00:13,  1.62it/s]

 76%|██████████████████████████████████████████████████████████████▏                   | 66/87 [00:41<00:13,  1.60it/s]

 77%|███████████████████████████████████████████████████████████████▏                  | 67/87 [00:42<00:12,  1.56it/s]

 78%|████████████████████████████████████████████████████████████████                  | 68/87 [00:43<00:12,  1.56it/s]

 79%|█████████████████████████████████████████████████████████████████                 | 69/87 [00:43<00:11,  1.59it/s]

 80%|█████████████████████████████████████████████████████████████████▉                | 70/87 [00:44<00:11,  1.49it/s]

 82%|██████████████████████████████████████████████████████████████████▉               | 71/87 [00:45<00:10,  1.55it/s]

 83%|███████████████████████████

True

In [ ]:
list_pinyin[1]

In [37]:
msn = """常德、渭南、孝感、丽水、运城、德州、张家口、鄂尔多斯、阳江、泸州、丹东、曲靖、乐山、许昌、湘潭、晋中、安阳、齐齐哈尔、北海、宝鸡、抚州、景德镇、延安、三明、抚顺亳州、日照、衢州、拉萨、淮北、焦作、平顶山、滨州、吉安、濮阳、眉山、池州、荆门、铜仁、长治、衡水、铜陵、承德、达州、邵阳、德阳、龙岩、南平、淮南、黄石、营口、东营、吉林、韶关、枣庄、怀化、宣城、临汾、聊城、梅州、盘锦、锦州、榆林、玉林、十堰、汕尾、咸宁、宜宾、永州、益阳、黔南州、黔东南、恩施、红河、大理、大同、鄂州、忻州、吕梁、黄山、开封、郴州、茂名、漯河、葫芦岛、河源、娄底、延边"""

In [38]:
import re
msn.replace('、','","')

'常德","渭南","孝感","丽水","运城","德州","张家口","鄂尔多斯","阳江","泸州","丹东","曲靖","乐山","许昌","湘潭","晋中","安阳","齐齐哈尔","北海","宝鸡","抚州","景德镇","延安","三明","抚顺亳州","日照","衢州","拉萨","淮北","焦作","平顶山","滨州","吉安","濮阳","眉山","池州","荆门","铜仁","长治","衡水","铜陵","承德","达州","邵阳","德阳","龙岩","南平","淮南","黄石","营口","东营","吉林","韶关","枣庄","怀化","宣城","临汾","聊城","梅州","盘锦","锦州","榆林","玉林","十堰","汕尾","咸宁","宜宾","永州","益阳","黔南州","黔东南","恩施","红河","大理","大同","鄂州","忻州","吕梁","黄山","开封","郴州","茂名","漯河","葫芦岛","河源","娄底","延边'

In [8]:

# for jack in a:
#     os.remove("../css破解/jack")
a

['873b6f82.woff', 'b6ce0b8f.woff', 'd1d35beb.woff']

In [12]:
os.chdir("D://360downloads//美团比赛//css破解//") #设置需要查看的目录D:\360downloads\美团比赛\css破解
a = [x for x in os.listdir()]
for jack in a:
    os.remove("D://360downloads//美团比赛//css破解//"+jack)
    

In [4]:
for x in range(0,len(list_pinyin)):
    url = 'http://www.dianping.com/'+ str(list_pinyin[x]) +'/ch10/g110'
    print(url)

http://www.dianping.com/kunming/ch10/g110
http://www.dianping.com/dalian/ch10/g110
http://www.dianping.com/xiamen/ch10/g110
http://www.dianping.com/hefei/ch10/g110
http://www.dianping.com/foshan/ch10/g110
http://www.dianping.com/fuzhou/ch10/g110
http://www.dianping.com/haerbin/ch10/g110
http://www.dianping.com/jinan/ch10/g110
http://www.dianping.com/wenzhou/ch10/g110
http://www.dianping.com/changchun/ch10/g110
http://www.dianping.com/shijiazhuang/ch10/g110
http://www.dianping.com/changzhou/ch10/g110
http://www.dianping.com/quanzhou/ch10/g110
http://www.dianping.com/nanning/ch10/g110
http://www.dianping.com/guiyang/ch10/g110
http://www.dianping.com/nanchang/ch10/g110
http://www.dianping.com/nantong/ch10/g110
http://www.dianping.com/jinhua/ch10/g110
http://www.dianping.com/xuzhou/ch10/g110
http://www.dianping.com/taiyuan/ch10/g110
http://www.dianping.com/jiaxing/ch10/g110
http://www.dianping.com/yantai/ch10/g110
http://www.dianping.com/huizhou/ch10/g110
http://www.dianping.com/baoding/ch

In [7]:
os.close(D://360downloads//美团比赛//css破解//)

SyntaxError: invalid syntax (<ipython-input-7-8681c6b3d1de>, line 1)